In [1]:
#Reference link: https://sagemaker-examples.readthedocs.io/en/latest/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.html

In [2]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client("sagemaker")

## You must have already run a hyperparameter tuning job to analyze it here.
## The Hyperparameter tuning jobs you have run are listed in the Training section on your SageMaker dashboard.
## Copy the name of a completed job you want to analyze from that list.
## For example: tuning_job_name = 'mxnet-training-201007-0054'.
tuning_job_names = ['pytorch-training-220906-0950', 'warmstart-220907-0317']

In [3]:
import pandas as pd

df_list = []
tuner_max_ranges = {}
for tuning_job_name in tuning_job_names:

    # run this cell to check current status of hyperparameter tuning job
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuning_job_name
    )

    status = tuning_job_result["HyperParameterTuningJobStatus"]
    if status != "Completed":
        print("Reminder: the tuning job has not been completed.")

    job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
    print("%d training jobs have completed" % job_count)

    objective = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]
    is_minimize = objective["Type"] != "Maximize"
    objective_name = objective["MetricName"]
    
    
    
    tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
    tuner_ranges = tuner.tuning_ranges
    if tuner_max_ranges == {}:
        tuner_max_ranges = tuner_ranges
    else:
        for item in tuner_max_ranges.keys():
            tuner_max_ranges[item]['MinValue'] = min(tuner_max_ranges[item]['MinValue'], tuner_ranges[item]['MinValue'])
            tuner_max_ranges[item]['MaxValue'] = max(tuner_max_ranges[item]['MaxValue'], tuner_ranges[item]['MaxValue'])
    
    full_df = tuner.dataframe()

    if len(full_df) > 0:
        df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
        if len(df) > 0:
            df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
            print("Number of training jobs with valid objective: %d" % len(df))
            print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
            pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
            df_list.append(df)
        else:
            print("No training jobs have reported valid results yet.")
            
df = pd.concat(df_list)
df

Reminder: the tuning job has not been completed.
29 training jobs have completed
Number of training jobs with valid objective: 30
{'lowest': 9.558319091796875, 'highest': 13.37951374053955}
Reminder: the tuning job has not been completed.
3 training jobs have completed
Number of training jobs with valid objective: 3
{'lowest': 9.873359680175781, 'highest': 10.951380729675293}


,dropout-rate,learning-rate,max-gradient-norm,minibatch-size,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
4,0.002674,0.000102,0.001049,248.0,pytorch-training-220906-0950-026-d663d12f,Completed,9.558319,2022-09-06 21:34:50+00:00,2022-09-06 22:10:47+00:00,2157.0
3,0.001703,0.000104,0.001008,248.0,pytorch-training-220906-0950-027-4dae5a55,Completed,9.560058,2022-09-06 22:17:53+00:00,2022-09-06 22:53:44+00:00,2151.0
9,0.001000,0.000100,0.001000,256.0,pytorch-training-220906-0950-021-511fe425,Completed,9.560384,2022-09-06 18:19:04+00:00,2022-09-06 18:56:34+00:00,2250.0
2,0.002691,0.000106,0.001045,244.0,pytorch-training-220906-0950-028-8bcd02c4,Completed,9.561897,2022-09-06 22:59:27+00:00,2022-09-06 23:34:55+00:00,2128.0
5,0.001428,0.000112,0.001019,236.0,pytorch-training-220906-0950-025-a9795511,Completed,9.564445,2022-09-06 20:55:26+00:00,2022-09-06 21:29:01+00:00,2015.0
1,0.003044,0.000103,0.001056,207.0,pytorch-training-220906-0950-029-a080ee15,Completed,9.564651,2022-09-06 23:41:15+00:00,2022-09-07 00:12:25+00:00,1870.0
7,0.001000,0.000100,0.001929,256.0,pytorch-training-220906-0950-023-7b6c86b8,Completed,9.568633,2022-09-06 19:35:49+00:00,2022-09-06 20:07:30+00:00,1901.0
13,0.003569,0.000139,0.001422,256.0,pytorch-training-220906-0950-017-871dfd3c,Completed,9.573410,2022-09-06 16:22:56+00:00,2022-09-06 16:53:27+00:00,1831.0
6,0.001080,0.000124,0.001473,254.0,pytorch-training-220906-0950-024-4f12c8ea,Completed,9.573951,2022-09-06 20:13:11+00:00,2022-09-06 20:47:47+00:00,2076.0
19,0.001000,0.000115,0.001000,195.0,pytorch-training-220906-0950-011-5cf5083f,Completed,9.574807,2022-09-06 13:32:55+00:00,2022-09-06 14:03:12+00:00,1817.0


In [4]:
from pprint import pprint
import numpy as np

lowest_rmse = np.inf
have_some_best_result = False
for tuning_job_name in tuning_job_names:
    # run this cell to check current status of hyperparameter tuning job
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuning_job_name
    )
    
    have_some_best_result = have_some_best_result | (tuning_job_result.get("BestTrainingJob", None) != None)
        

    tuner_best_rmse = tuning_job_result["BestTrainingJob"]['FinalHyperParameterTuningJobObjectiveMetric']['Value']
    if tuner_best_rmse < lowest_rmse:
        best_model = tuning_job_result["BestTrainingJob"]
        best_tuner = tuning_job_name
        lowest_rmse = tuner_best_rmse
        
        
if have_some_best_result:
    print("Best tuner found so far: {}".format(best_tuner))
    print("Best model found so far:")
    pprint(best_model)
else:
    print("No training jobs have reported results yet.")

Best tuner found so far: pytorch-training-220906-0950
Best model found so far:
{'CreationTime': datetime.datetime(2022, 9, 6, 21, 32, 49, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'val:RMSE',
                                                 'Value': 9.558319091796875},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2022, 9, 6, 22, 10, 47, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:551329315830:training-job/pytorch-training-220906-0950-026-d663d12f',
 'TrainingJobName': 'pytorch-training-220906-0950-026-d663d12f',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2022, 9, 6, 21, 34, 50, tzinfo=tzlocal()),
 'TunedHyperParameters': {'dropout-rate': '0.002674059547789765',
                          'learning-rate': '0.0001021695939710504',
                          'max-gradient-norm': '0.0010488370375433596',
                          'minibatch-size': '248'}}


In [10]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuner)

# p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
# p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
# show(p)

Loading BokehJS ...

In [11]:
ranges = tuner_max_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))